In [2]:
import pandas as pd


# 데이터 정의
data = pd.read_csv("0.원본+전처리_데이터/TB_SHC_SALE_CUST_INFO_202404.csv")

# 데이터프레임 생성
df = pd.DataFrame(data)

# 각 시별 카드소비데이터 전처리

In [3]:
#수원시
# 	•	장안구 (4111100000)
# 	•	권선구 (4111300000)
# 	•	팔달구 (4111500000)
# 	•	영통구 (4111700000)

# 용인시 → 처인, 기흥, 수지 (3개)
# 	•	처인구 (4146100000)
# 	•	기흥구 (4146300000)
# 	•	수지구 (4146500000)

# 부천시 → 원미, 소사, 오정(3개)
# 	◦	원미구 (4119200000)
# 	◦	소사구 (4119400000)
# 	◦	오정구 (4119600000)

# 시흥시 (4139000000)→ 없음(행정동만 20개)

# 광명시(4121000000) → 없음(행정동만 19개)

# 안산시 → 단원구,상록구(2개)
# 	◦	단원구(4127300000)
# 	◦	상록구(4127100000)

# 안양시 → 만안, 동안 (2개)
# 	◦	만안구 (4117100000)
# 	◦	동안구 (4117300000)

# 포천시 (4165000000) → 없음(읍면동 14개)

# 하남시 (4145000000) → 없음(행정동 14개)

# 화성시 (4159000000) → 없음(법정동 25개)

In [4]:
suwon_cd = [4111100000,4111300000,4111500000,4111700000]
yong_cd = [4146100000,4146300000,4146500000]
bu_cd = [4119200000,4119400000,4119600000]
si_cd = [4139000000]
gwang_cd = [4121000000] 
ansan_cd = [4127300000, 4127100000]
anyang_cd = [4117100000,4117300000]
po_cd = [4165000000]
hanam_cd = [4145000000]
hwasung_cd = [4159000000]

In [6]:
df_cd = df[df['시군구코드'].isin(yong_cd)] 

In [8]:
df_cd = df_cd[df_cd['십단위연령구분코드'] == 6] # 연령대 분류 60대 이상
df_cd = df_cd[(df_cd['업종대분류코드'] == 2) | (df_cd['업종대분류코드'] == 4) | (df_cd['업종대분류코드'] == 5) | (df_cd['업종중분류코드']==72)] # 소비코드 분류 2번: 소매/유통 7번 중
# 가전제품, 건강/기호식품, 선물/완구, 스포츠/레저용품, 음식료품소매, 의복/의류, 인테리어/가정용품, 종합소매점,  

In [9]:
df_cd = df_cd.iloc[:, [3,4,5,15]] # df 가져오기

In [10]:
df_cd['업종대분류코드'] = df_cd['업종대분류코드'].map({7:'의료/건강',2:'소매/유통',4:'소매/유통',5:'소매/유통'}) # 칼럼 명 설정
# df_cd['업종대분류코드'].value_counts()

# 나갈 형태 만들어주기

In [11]:
df_group = df_cd.groupby(['읍면동코드','업종대분류코드']).agg({'결제건수':'sum'}).reset_index() #
df_pivot = df_group.pivot_table(columns='업종대분류코드',index='읍면동코드',values='결제건수')
df_pivot = df_pivot.fillna(0)
# df_pivot

In [12]:
df_pivot.to_csv('0.원본+전처리_데이터/데이터구역_카드소비데이터_수원.csv')

# 한번에 진행

In [11]:
def preprocessing_ansim(df, cd, name):
    df_cd = df[df['시군구코드'].isin(cd)] 
    df_cd = df_cd[df_cd['십단위연령구분코드'] == 6] # 연령대 분류 60대 이상
    df_cd = df_cd[(df_cd['업종대분류코드'] == 2) | (df_cd['업종대분류코드'] == 4) | (df_cd['업종대분류코드'] == 5) | (df_cd['업종중분류코드']==72)] # 소비코드 분류 2번: 소매/유통 7번 중
    df_cd = df_cd.iloc[:, [3,4,5,15]] # df 가져오기
    df_cd['업종대분류코드'] = df_cd['업종대분류코드'].map({7:'의료/건강',2:'소매/유통',4:'소매/유통',5:'소매/유통'}) # 칼럼 명 설정
    df_cd['업종대분류코드'].value_counts()
    df_group = df_cd.groupby(['읍면동코드','업종대분류코드']).agg({'결제건수':'sum'}).reset_index() #
    df_pivot = df_group.pivot_table(columns='업종대분류코드',index='읍면동코드',values='결제건수')
    df_pivot = df_pivot.fillna(0)
    df_pivot.to_csv(f'전처리_데이터/데이터구역_카드소비데이터_{name}.csv')
    return

In [12]:
lst_cd = [suwon_cd,yong_cd,bu_cd,si_cd,gwang_cd,ansan_cd,anyang_cd,po_cd,hanam_cd,hwasung_cd]
lst_name = ['수원','용인','부천','시흥','광명','안산','안양','포천','하남','화성']

In [13]:
for cd, name in zip(lst_cd,lst_name):
    preprocessing_ansim(df, cd, name)